In [ ]:
# ============================================================================
#
# tokenizer_segmentation_scores.ipynb
# Here is a complete and user-friendly Jupyter Notebook designed for language teams and developers to:
# ✅ Run tokenizer segmentation tests from a .csv
# ✅ Compute precision, recall, and F1 score
# ✅ Visualize token-by-token comparison
# ✅ Highlight mismatches for easy human review
# ✅ Export results to .ods (LibreOffice) and .html formats
#
# Author: 
#   MoniGarr (Monica Peters), monigarr@MoniGarr.com
#
# This repository supports language revival & retention for
#     Polysynthetic, Low-Resource Indigenous Languages that
#       might lack industry standard language ISO codes.
#
# License: Apache 2.0
# 
# For technical consulting, collaboration, or mentorship on Indigenous
# Language Revival & Retention Tech Solutions (AI, XR, 3D, Cultural Protocols)
# contact:
#   MoniGarr (Monica Peters) – monigarr@monigarr.com
#   Founder of MoniGarr.com LLC and MohawkLanguage.ca
#   Akwesasne-based Onkwehonwe (Indigenous, Kanien’kéhake, Mohawk of Akwesasne)
#   https://www.linkedin.com/in/3dtechartist
#
# ============================================================================

In [ ]:
🧠 Features Summary
Feature	                            Included ✅
Token-by-token comparison	                ✅
Precision, Recall, F1 per sentence	        ✅
Visual diff highlighting (HTML)	            ✅
ODS export for offline Elders	            ✅
HTML export for easy review	                ✅

In [ ]:
# 📓 tokenizer_segmentation_scores.ipynb

import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
import difflib
import os

In [ ]:
# 🔧 CONFIG
csv_path = "../tokenizer/tests/tokenizer_test_set.csv"
tokenizer_path = "../tokenizer/custom_tokenizer.json"


In [ ]:
# 📚 Load data
df = pd.read_csv(csv_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [ ]:
# URL encode helper
from urllib.parse import quote

In [ ]:
# 📊 Prepare result table
github_org = "monigarr""
github_repo = "mini-indig-llm-kit"

results = []


for _, row in df.iterrows():
    case_id = row["id"]
    dialect = row["dialect"]
    input_text = row["input"]
    expected_tokens = [tok.strip() for tok in row["expected_tokens"].split(",")]
    predicted_tokens = tokenizer.tokenize(input_text)

    # Scoring
    true_set = set(expected_tokens)
    pred_set = set(predicted_tokens)
    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

    # 🔍 Diff with HTML highlight
    matcher = difflib.SequenceMatcher(None, expected_tokens, predicted_tokens)
    diff_html = ""
    for opcode, a0, a1, b0, b1 in matcher.get_opcodes():
        if opcode == 'equal':
            diff_html += " ".join([f"<span style='color:green'>{tok}</span>" for tok in expected_tokens[a0:a1]]) + " "
        else:
            diff_html += " ".join([f"<span style='color:red'>{tok}</span>" for tok in expected_tokens[a0:a1]]) + " "
            diff_html += " ".join([f"<span style='color:orange'>{tok}</span>" for tok in predicted_tokens[b0:b1]]) + " "

    # 📝 Add annotation link
    issue_title = f"Annotation for Test {case_id}"
    issue_body = (
        f"**Input:** {input_text}\n"
        f"**Expected Tokens:** {', '.join(expected_tokens)}\n"
        f"**Predicted Tokens:** {', '.join(predicted_tokens)}\n"
        f"**Dialect:** {dialect}"
    )
    issue_url = (
        f"https://github.com/{github_org}/{github_repo}/issues/new?"
        f"title={quote(issue_title)}&body={quote(issue_body)}"
    )
    annotation_link = f"""<br><a href="{issue_url}" target="_blank" rel="noopener" style="font-size:small;">
    📝 Suggest correction</a>"""

    results.append({
        "ID": case_id,
        "Dialect": dialect,
        "Input": input_text,
        "Expected": ", ".join(expected_tokens),
        "Predicted": ", ".join(predicted_tokens),
        "Diff Highlight": diff_html + annotation_link,
        "Precision": round(precision, 2),
        "Recall": round(recall, 2),
        "F1": round(f1, 2)
    })

results_df = pd.DataFrame(results)

In [ ]:
# 📈 Display Scores
results_df[["ID", "Dialect", "Precision", "Recall", "F1"]]

In [ ]:
from IPython.display import display, HTML

# Display all annotated rows
display(HTML(results_df[["ID", "Input", "Diff Highlight"]].to_html(escape=False, index=False)))

In [ ]:
# 🖼️ HTML Render
from IPython.display import display, HTML
display(HTML(results_df[["ID", "Input", "Diff Highlight"]].to_html(escape=False)))

In [ ]:
# 🧾 EXPORTS

# Plain CSV for tech review
results_df.to_csv("../tokenizer/tests/segmentation_results.csv", index=False)

# LibreOffice Version for Language Teams
results_df.to_excel("../tokenizer/tests/segmentation_results.ods", index=False)

# Visual diff to share / view in browser
results_df.to_html("../tokenizer/tests/segmentation_results.html", escape=False, index=False)

print("✅ Exported results to CSV, ODS, and HTML formats.")